# Cluster comparison
This will be accomplished in two steps: 
 1. statistical test across each variable comparing the sample populations across various measurements 
 2. identifying the mean values across each of these measurements.  
 
The result will be a report for each patient subgroup displaying its expected values for a particular measurement, the expected values for the same measurement for the rest of the patient population, the difference, and whether or not that difference is statistically significant. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import pandas as pd
import numpy as np

# Statistical tests
from scipy.stats import ttest_ind, chisquare, chi2_contingency

# user scripts
from dfGenUtils import convertCategorical, genPatientDF, dropMissings

In [3]:
input_dir = "../data/heorData/"
graph_dir = "../data/graphs/"
patient_dir = "../data/patientData/"
output_dir = "../data/heorData/results/"

## Generate Dataframe and Metadata Variables
  1. Need to compute descriptive stats and statistical tests before imputation of missing vars
  2. Need to identify which columns belong to which data subgroups

In [4]:
cluster_df = pd.read_csv(graph_dir + "cosine_cluster5.csv")
cluster_df.head()

,subject_id,cluster
0,4,0
1,314,0
2,761,0
3,1152,0
4,3064,0


In [5]:
mortality = pd.read_csv(patient_dir + "mortality.csv")
mortality.head()

,subject_id,last_admission,death_date,time2event,mortality30
0,7275,2140-04-12,2140-05-08,26.0,1
1,357,2199-12-21,2201-08-02,589.0,0
2,878,2137-11-24,2137-12-11,17.0,1
3,794,2190-11-29,2191-09-19,294.0,0
4,1457,2189-01-25,NaN,NaN,0


In [6]:
# generate our patient dataframe and drop missings. But, do not impute
files = os.listdir(patient_dir)
files = [f for f in files if re.search("chart events|cross sect", f)]

patient_df = genPatientDF(patient_dir, files, cluster_df, mortality, time2event = True)
#patient_df = dropMissings(patient_df, .75)
patient_df.head()

,subject_id,activity,activity_tolerance,braden_activity,braden_mobility,family_communication,marital_status,race,gender,appetite,...,min_protein,max_protein,avg_protein,avg_tot_protein,avg_urine_prot,avg_ascite_prot,avg_fetoprotein,cluster,mortality30,time2event
0,4,Bedrest,Tolerated Well,Walks Occasionally,Slight Limitations,Family Visited,Single,NaN,F,NaN,...,30.0,30.0,30.0,NaN,NaN,NaN,NaN,0,0,NaN
1,52,Bedrest,Tolerated Well,Bedfast,Slight Limitations,Family Visited,Single,NaN,M,NaN,...,NaN,NaN,NaN,NaN,18.0,NaN,NaN,0,0,548.0
2,78,Bedrest,Tolerated Well,Bedfast,No Limitations,Family Visited,A,NaN,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1083.0
3,117,Bedrest,Tolerated Well,Bedfast,Very Limited,Family Visited,Single,NaN,F,NaN,...,500.0,500.0,500.0,5.05,145.0,NaN,NaN,0,1,18.0
4,140,Bedrest,Tolerated Well,Chairfast,No Limitations,Family Called,Divorced,NaN,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,677.0


In [7]:
list(patient_df.columns)

['subject_id',
 'activity',
 'activity_tolerance',
 'braden_activity',
 'braden_mobility',
 'family_communication',
 'marital_status',
 'race',
 'gender',
 'appetite',
 'braden_nutrition',
 'diet_type',
 'special_diet',
 'heart_rhythm',
 'lll_lung_sounds',
 'lul_lung_sounds',
 'rll_lung_sounds',
 'rul_lung_sounds',
 'respiratory_effort',
 'respiratory_pattern',
 'mental_status',
 'recreational_drug_use',
 'pain_cause',
 'pain_location',
 'pain_present',
 'pain_type',
 'abdominal_assessment',
 'bowel_sounds',
 'braden_moisture',
 'cough_reflex',
 'gag_reflex',
 'oral_cavity',
 'skin_color',
 'skin_condition',
 'speech',
 'age',
 'max_heart_rate',
 'min_heart_rate',
 'avg_heart_rate',
 'max_resp_rate',
 'min_resp_rate',
 'avg_resp_rate',
 'anemia',
 'angina',
 'arrhythmias',
 'asthma',
 'cad',
 'chf',
 'copd',
 'cva',
 'diabetes_-_insulin',
 'diabetes_-_oral_agent',
 'etoh',
 'gi_bleed',
 'hemo_or_pd',
 'hepatitis',
 'hypertension',
 'liver_failure',
 'mi',
 'pvd',
 'pacemaker',
 'pancre

In [8]:
num_clusters = max(cluster_df.cluster) +1
clusters = range(num_clusters)
num_clusters

2

In [9]:
# identify which columns belong to which data subgroups
# format is dictionary{column name: data_group}
def extractColumnGroupings(patient_dir):
    columnMapping = {}
    patient_files = os.listdir(patient_dir)
    
    manualMap = {"heart lung rate": "heart lung",
                 "pain level": "pain",
                 "weight": "diet",
                 "mental drug": "medical history"}
    
    for f in patient_files:
        curr_df = pd.read_csv(patient_dir + f)
        columns = curr_df.columns
        
        # extract data subgroup name 
        m = re.search("_.*_(.*?)\.", f)
        if m:
            data_subgroup = m.group(1)
            
            # chart event files have the column names in the label variable
            chart_event = re.search("chart events_categorical", f)
            if chart_event:
                columns = list(curr_df.label.unique())
            
            if chart_event or data_subgroup == "pain level":
                columns = ['_'.join(c for c in col.lower().split()) for col in columns]
                
            # manual mapping of some
            try:
                data_subgroup = manualMap[data_subgroup]
            except KeyError:
                pass
            
                
            [columnMapping.update({col:data_subgroup}) for col in columns]
            
    # manually add mortality
    columnMapping['mortality30'] = 'mortality'
    columnMapping['time2event'] = 'mortality'
    
    return columnMapping

columnMapping = extractColumnGroupings(patient_dir)
columnMapping

{'activity': 'activity',
 'activity_tolerance': 'activity',
 'braden_activity': 'activity',
 'braden_mobility': 'activity',
 'family_communication': 'demographics',
 'marital_status': 'demographics',
 'gender': 'demographics',
 'race': 'demographics',
 'braden_nutrition': 'diet',
 'diet_type': 'diet',
 'appetite': 'diet',
 'special_diet': 'diet',
 'heart_rhythm': 'heart lung',
 'lll_lung_sounds': 'heart lung',
 'lul_lung_sounds': 'heart lung',
 'rll_lung_sounds': 'heart lung',
 'rul_lung_sounds': 'heart lung',
 'respiratory_pattern': 'heart lung',
 'respiratory_effort': 'heart lung',
 'mental_status': 'medical history',
 'recreational_drug_use': 'medical history',
 'pain_location': 'pain',
 'pain_present': 'pain',
 'pain_cause': 'pain',
 'pain_type': 'pain',
 'abdominal_assessment': 'physical assessment',
 'bowel_sounds': 'physical assessment',
 'braden_moisture': 'physical assessment',
 'oral_cavity': 'physical assessment',
 'skin_color': 'physical assessment',
 'skin_condition': 'phy

In [10]:
# export the mapping to csv so that we can work with it in R
mappingR = pd.DataFrame.from_dict(columnMapping, orient = "index").reset_index()
mappingR.columns = ['column','subgroup']
mappingR.to_csv("../data/subgroupMapping.csv", index = False)

In [11]:
# define our variable groups - maybe I subset to top 50 most important columns? 
# I will need to extract the groupings 
num_vars = patient_df.select_dtypes(include=np.number).columns.tolist()
num_vars = [c for c in num_vars if c not in ["subject_id", "cluster"]]

cat_vars = patient_df.select_dtypes(include=["object"]).columns.tolist()
num_vars, cat_vars

(['age',
  'max_heart_rate',
  'min_heart_rate',
  'avg_heart_rate',
  'max_resp_rate',
  'min_resp_rate',
  'avg_resp_rate',
  'anemia',
  'angina',
  'arrhythmias',
  'asthma',
  'cad',
  'chf',
  'copd',
  'cva',
  'diabetes_-_insulin',
  'diabetes_-_oral_agent',
  'etoh',
  'gi_bleed',
  'hemo_or_pd',
  'hepatitis',
  'hypertension',
  'liver_failure',
  'mi',
  'pvd',
  'pacemaker',
  'pancreatitis',
  'renal_failure',
  'seizures',
  'smoker',
  'abdominal',
  'back',
  'both_legs',
  'chest_pain',
  'generalized',
  'headache',
  'incisional',
  'jaw',
  'left_arm',
  'left_chest',
  'left_elbow',
  'left_flank',
  'left_foot',
  'left_hip',
  'left_leg',
  'left_lower_quad',
  'left_lower_quadrant',
  'left_shoulder',
  'left_upper_quad',
  'left_upper_quadrant',
  'mediastinal',
  'midscapular',
  'neck',
  'not_indicated',
  'perineum',
  'periumbilical',
  'right_arm',
  'right_chest',
  'right_elbow',
  'right_flank',
  'right_foot',
  'right_hip',
  'right_leg',
  'right_l

## Statistical Tests
I take a one vs. all approach here. For each cluster, I compare the samples from the selected cluster to all other data. I do this for each cluster.

For numeric data, I compare the means using a t-test. For categorical data, I compare using the chi-squared test. 

In [12]:
# run ttests for all numeric variables for a given cluster
def tTests(cols, patient_df, cluster):
    test_res = []
    for col in cols:
        main = patient_df.loc[patient_df.cluster == cluster, col].dropna()
        other = patient_df.loc[patient_df.cluster != cluster, col].dropna()

        # run tests and append
        res = ttest_ind(main, other)
        sum_stats_main = {"mean":round(main.mean(),2),
                          "std":round(main.std(),2)}
        sum_stats_other = {"mean":round(other.mean(),2),
                           "std":round(other.std(),2)}
        
        test_res.append([col, # column
                         sum_stats_main,# numeric summary stats for comparison cluster                         
                         sum_stats_other, # numeric summary stats for all others
                         round(res.pvalue,4)]) # p value
    return test_res 

#numeric_res = tTests(num_vars, patient_df, c)
#numeric_res[:5]

In [13]:
# extract the top three most frequent categorical values and their percent frequency
def mostFrequentCats(frequencies, n = 3):
    top_cats = {}
    total_obs = frequencies.sum()
    freq_ratio = frequencies/total_obs
    [top_cats.update({row[0]: round(row[1],2)}) for i, row in enumerate(freq_ratio.iteritems()) if i < n]
    return top_cats

In [14]:
# run our chisquared tests for all categorical variables leveraging contingency tables for a given cluster
def chi2Tests(cols, patient_df, cluster):
    test_res = []
    for col in cols:
        # generate our contingency table
        main = patient_df.loc[patient_df.cluster == cluster, col].value_counts()
        other = patient_df.loc[patient_df.cluster != cluster, col].value_counts()
        contingency_tbl = pd.merge(main, other, how = 'left', left_index = True, right_index = True).fillna(0)
        
        # run tests and append
        chi2, p, ddof, expected = chi2_contingency(contingency_tbl)
        test_res.append([col, # column
                         mostFrequentCats(main,3), # top 3 cat labels and their frequencies
                         mostFrequentCats(other,20), # top 3 labels for the other group
                         round(p, 4)]) # p value
    return test_res

#cat_res = chi2Tests(cat_vars, patient_df, c)
#cat_res[:5]


## Generate the Report


In [15]:
def genReport(num_res, cat_res, columnMapping, cluster):
    # gen base report
    num_df = pd.DataFrame(num_res)
    num_df['data_type'] = "numeric"
    cat_df = pd.DataFrame(cat_res)
    cat_df['data_type'] = "categorical"
    
    report = pd.concat([num_df, cat_df], axis = 0)
    report.columns = ['variable', 'main_summary_stats', 'other_summary_stats', 'p_value', 'data_type']
    report['cluster'] = cluster
    
    # extract the data subgroups
    report['data_subgroup'] = report.variable.map(columnMapping)
    
    return report.reset_index(drop = True)

#report = genReport(numeric_res, cat_res, columnMapping, c)
#report.head()

In [16]:
# run through the process for both clusters and then concatenate
cluster_report_list = []
for c in range(num_clusters):
    # run tests
    num_res = tTests(num_vars, patient_df, c)
    cat_res = chi2Tests(cat_vars, patient_df, c)
    
    # gen report
    report = genReport(num_res, cat_res, columnMapping, c)
    cluster_report_list.append(report)
    
# concatenate
cluster_reports = pd.concat(cluster_report_list, axis = 0)
cluster_reports

,variable,main_summary_stats,other_summary_stats,p_value,data_type,cluster,data_subgroup
0,age,"{'mean': 58.56, 'std': 25.93}","{'mean': 59.82, 'std': 28.15}",0.2121,numeric,0,demographics
1,max_heart_rate,"{'mean': 120.64, 'std': 24.47}","{'mean': 6866.49, 'std': 259759.05}",0.3342,numeric,0,heart lung
2,min_heart_rate,"{'mean': 59.98, 'std': 15.74}","{'mean': 63.29, 'std': 15.94}",0.0000,numeric,0,heart lung
3,avg_heart_rate,"{'mean': 86.18, 'std': 13.06}","{'mean': 96.43, 'std': 373.31}",0.3076,numeric,0,heart lung
4,max_resp_rate,"{'mean': 34.67, 'std': 13.31}","{'mean': 38.29, 'std': 123.36}",0.2779,numeric,0,heart lung
...,...,...,...,...,...,...,...
181,gag_reflex,"{'Intact': 0.76, 'Impaired': 0.2, 'Absent': 0.04}","{'Intact': 0.68, 'Impaired': 0.22, 'Absent': 0...",0.0001,categorical,1,physical assessment
182,oral_cavity,"{'Teeth/Tissue WNL': 0.91, 'Bleeding Gum': 0.0...","{'Teeth/Tissue WNL': 0.94, 'Bleeding Gum': 0.0...",0.0000,categorical,1,physical assessment
183,skin_color,"{'Normal for Race': 0.75, 'Jaundiced': 0.19, '...","{'Normal for Race': 0.58, 'Jaundiced': 0.32, '...",0.0000,categorical,1,physical assessment
184,skin_condition,"{'Dry ': 0.99, 'Dry': 0.01, 'Diaphoretic': 0.0}","{'Dry': 0.69, 'Dry ': 0.3, 'Clammy': 0.01, 'Di...",0.0000,categorical,1,physical assessment


In [17]:
cluster_reports.to_csv(output_dir + "Cluster Reports (cosine_knn5_k2).csv")

## Split the reports for visualization


In [18]:
numeric_report = cluster_reports.loc[cluster_reports['data_type'] == "numeric"]
categorical_report = cluster_reports.loc[cluster_reports['data_type'] == "categorical"]

In [19]:
# cleanup numeric
numeric_report['cluster_mean'] = numeric_report['main_summary_stats'].apply(lambda x: x['mean'])
numeric_report['cluster_std'] = numeric_report['main_summary_stats'].apply(lambda x: x['std'])
numeric_report['other_mean'] = numeric_report['other_summary_stats'].apply(lambda x: x['mean'])
numeric_report['other_std'] = numeric_report['other_summary_stats'].apply(lambda x: x['std'])
numeric_report = numeric_report.drop(columns = ["main_summary_stats", "other_summary_stats",
                                               "data_type"])

#reorder
numeric_report = numeric_report[['variable', 'cluster_mean', 'other_mean', 
                                 'cluster_std', 'other_std', 'p_value', 
                                 'data_subgroup', 'cluster']]
numeric_report

C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,variable,cluster_mean,other_mean,cluster_std,other_std,p_value,data_subgroup,cluster
0,age,58.56,59.82,25.93,28.15,0.2121,demographics,0
1,max_heart_rate,120.64,6866.49,24.47,259759.05,0.3342,heart lung,0
2,min_heart_rate,59.98,63.29,15.74,15.94,0.0000,heart lung,0
3,avg_heart_rate,86.18,96.43,13.06,373.31,0.3076,heart lung,0
4,max_resp_rate,34.67,38.29,13.31,123.36,0.2779,heart lung,0
...,...,...,...,...,...,...,...,...
147,avg_urine_prot,133.81,127.10,263.85,282.10,0.8095,proteins,1
148,avg_ascite_prot,3.18,1.34,24.12,0.92,0.1844,proteins,1
149,avg_fetoprotein,2592.43,4173.44,29496.35,65755.93,0.7458,proteins,1
150,mortality30,0.24,0.31,0.43,0.46,0.0001,mortality,1


In [20]:
# clean up categorical 
def indexKeys(keys, index):
    try:
        return list(keys)[index]
    except IndexError:
        return np.nan
    
def extractValue(d, key):
    try:
        return d[key]
    except KeyError:
        return np.nan

# first level
categorical_report['primary_label'] = categorical_report['main_summary_stats'].apply(lambda x: indexKeys(x.keys(),0))
categorical_report['primary_cluster_value'] = categorical_report.apply(lambda x: x.main_summary_stats[x.primary_label], axis = 1)
categorical_report['primary_other_value'] = categorical_report.apply(lambda x: x.other_summary_stats[x.primary_label], axis = 1)

# second level
categorical_report['secondary_label'] = categorical_report['main_summary_stats'].apply(lambda x: indexKeys(x.keys(),1))
categorical_report['secondary_cluster_value'] = categorical_report.apply(lambda x: x.main_summary_stats[x.secondary_label], axis = 1)
categorical_report['secondary_other_value'] = categorical_report.apply(lambda x: x.other_summary_stats[x.secondary_label], axis = 1)

# third level
categorical_report['tertiary_label'] = categorical_report['main_summary_stats'].apply(lambda x: indexKeys(x.keys(),2))
categorical_report['tertiary_cluster_value'] = categorical_report\
                    .apply(lambda x: extractValue(x.main_summary_stats, x.tertiary_label), axis = 1)
categorical_report['tertiary_other_value'] = categorical_report\
                    .apply(lambda x: extractValue(x.other_summary_stats, x.tertiary_label), axis = 1)

# reorder and drop
categorical_report = categorical_report[["variable", 
                                         "primary_label", "primary_cluster_value", "primary_other_value",
                                         'secondary_label', 'secondary_cluster_value', 'secondary_other_value',
                                         'tertiary_label', 'tertiary_cluster_value', 'tertiary_other_value', 
                                         'p_value', 'data_subgroup', 'cluster']]

categorical_report

C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\ABell\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,variable,primary_label,primary_cluster_value,primary_other_value,secondary_label,secondary_cluster_value,secondary_other_value,tertiary_label,tertiary_cluster_value,tertiary_other_value,p_value,data_subgroup,cluster
152,activity,Bedrest,0.99,0.98,Chair,0.00,0.01,Commode,0.00,0.01,0.4861,activity,0
153,activity_tolerance,Tolerated Well,0.93,0.80,Good,0.04,0.18,Fair,0.02,0.02,0.0000,activity,0
154,braden_activity,Bedfast,0.92,0.90,Chairfast,0.04,0.05,Walks Occasionally,0.04,0.04,0.3649,activity,0
155,braden_mobility,Slight Limitations,0.49,0.55,Very Limited,0.37,0.32,Completely Immobile,0.07,0.06,0.0041,activity,0
156,family_communication,Family Visited,0.67,0.50,Family Called,0.27,0.18,Family Talked to MD,0.03,0.05,0.0000,demographics,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,gag_reflex,Intact,0.76,0.68,Impaired,0.20,0.22,Absent,0.04,0.09,0.0001,physical assessment,1
182,oral_cavity,Teeth/Tissue WNL,0.91,0.94,Bleeding Gum,0.03,0.03,Dentures,0.02,0.00,0.0000,physical assessment,1
183,skin_color,Normal for Race,0.75,0.58,Jaundiced,0.19,0.32,Pale,0.04,0.06,0.0000,physical assessment,1
184,skin_condition,Dry,0.99,0.30,Dry,0.01,0.69,Diaphoretic,0.00,0.00,0.0000,physical assessment,1


In [21]:
# export
numeric_report.to_csv(output_dir + "Numeric Report Cleaned (cosine_knn5_k2).csv", index = False)
categorical_report.to_csv(output_dir + "Categorical Report Cleaned (cosine_knn5_k2).csv", index = False)